In [1]:

from IPython import embed

In [2]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import cv2


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.nn.functional as F



In [4]:

trainTransform  = torchvision.transforms.Compose([torchvision.transforms.Grayscale(num_output_channels=1),
                                    torchvision.transforms.ToTensor()])
batch_size = 16

trainset_rgb = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=ToTensor())
trainset_gray = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=trainTransform)
train_rgb = torch.utils.data.DataLoader(trainset_rgb, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
train_gray = torch.utils.data.DataLoader(trainset_gray, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset_gray = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=trainTransform)
test_gray = torch.utils.data.DataLoader(trainset_gray, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True)#, transform=ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)



classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


Auto-Encoder



In [5]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(32*32*3, 512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 32*32*3),
            nn.Sigmoid()
            
        )

    def forward(self, x):
        img_encoded = self.encoder(x)
        img_decoded = self.decoder(img_encoded)
        return img_encoded,img_decoded
    
autoencoder = AutoEncoder()

In [6]:
import torch.optim as optim

cross_entropy_loss = nn.CrossEntropyLoss()
MSE_loss = torch.nn.MSELoss()
optimizer_SGD = optim.SGD(autoencoder.parameters(), lr=0.001, momentum=0.5)
optimizer = torch.optim.Adam(autoencoder.parameters(),
                             lr = 1e-4,
                             weight_decay = 1e-5)

In [8]:

epochs = 100

# New Section

In [9]:

for epoch in range(epochs):  # loop over the dataset multiple times
    encoded_images = []
    done = True
    running_loss = 0.0
    for img,label in train_rgb:
        img = img.reshape(-1,32*32*3)

        encoded_img, output_im = autoencoder(img)
        encoded_images.append(encoded_img)

        # zero the parameter gradients
        optimizer.zero_grad()

        loss = MSE_loss(output_im, img)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if epoch % 1 == 0 and done:    
            print('[%5d] loss: %.10f' %
                  (epoch + 35 , running_loss / 10))
            running_loss = 0.0
            done = False
        

print('Finished Training')

[   35] loss: 0.0027415281
[   36] loss: 0.0023557618
[   37] loss: 0.0028119782


KeyboardInterrupt: 

In [11]:
# PATH_1 = './Cifar_34_c_53_w.pth'
# opt_path = './Cifar_34_c_53_w_opt.pth'
# torch.save(autoencoder.state_dict(), PATH_1)
# torch.save(optimizer.state_dict(), opt_path)


In [12]:
from matplotlib import pyplot as plt
%matplotlib inline
for i,item in enumerate(img):
#     print(item.shape)
    item = item.reshape(3,32,32)
    item = item.permute(1, 2, 0)
    plt.imsave('{}.jpg'.format(i), item.numpy())#, cmap='gray')
print(output_im.shape)
# item_old = torch.tensor([0])
for i,item in enumerate(output_im):
    print(item.shape)
    item = item.reshape(3,32,32)
    item = item.permute(1, 2, 0)
    plt.imsave('{}_r.jpg'.format(i), item.detach().numpy())#, cmap='gray')
    if i>0 and torch.equal(item,item_old):
        print('problem')
    item_old = item
    



# plt.imshow(item[0])
# plt.savefig('foo.png')
# plt.show()
# cv2.imwrite('k.jpg',item[0].numpy())

torch.Size([16, 3072])
torch.Size([3072])
torch.Size([3072])
torch.Size([3072])
torch.Size([3072])
torch.Size([3072])
torch.Size([3072])
torch.Size([3072])
torch.Size([3072])
torch.Size([3072])
torch.Size([3072])
torch.Size([3072])
torch.Size([3072])
torch.Size([3072])
torch.Size([3072])
torch.Size([3072])
torch.Size([3072])


In [23]:
encoded_images = []
for img,label in train_rgb:
    img = img.reshape(-1,32*32*3)
#         print(img.shape)
    encoded_img, output_im = autoencoder(img)
    for i,item in enumerate(encoded_img):
      encoded_images.append(item.detach().numpy())
encoded_images = np.array(encoded_images)

In [24]:
from sklearn.cluster import KMeans

skmean = KMeans(
    n_clusters=3, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)

In [25]:
predict = skmean.fit_predict(encoded_images)

In [26]:
from sklearn.metrics import silhouette_samples, silhouette_score
silhouette_score(encoded_images,predict)

0.26556563